<a href="https://colab.research.google.com/github/alexaziegler/forecast-predictive-model/blob/main/synthetic_data_for_forecast_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objective: Use Gretel Data Designer to generate a realistic synthetic dataset for training forecast model

# Step 0: If you don't already have one, create an account on Gretel. Make sure to generate your API key.



There are 50 free credits, I completed the "getting started" notebook and only used 0.06 credit.



---




# Step 1: Install Gretel client


There was a compatibility error so also installing pyarrow

In [ ]:
pip install -U gretel_client pyarrow

  Using cached pyarrow-20.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)




---



# Step 2: Initialize data designer and input API key

In [ ]:
from gretel_client.navigator_client import Gretel
import gretel_client.data_designer.params as P
import gretel_client.data_designer.columns as C
gretel = Gretel(api_key="prompt")

Gretel API Key: ··········
Logged in as alexaziegler01@gmail.com ✅


INFO:gretel_client.navigator_client:Using project: default-sdk-project-5c832794c08e0de
INFO:gretel_client.navigator_client:Project link: https://console.gretel.ai/proj_2xVzJtcQwSxUcXls5h9QS4Igjx5




---



# Step 3: Set the Model Suite which is a collection of models grouped by license permissiveness

From what I could find, "apache-2.0" provides access to Mistral, Qwen and others whereas Azure, Llama, and Gemini required extra steps/stricter access. Since I'm still new to this, I'll just stick with the fully permissive "apache-2.0" option.

In [ ]:
# Initialize a new Data Designer instance using the `data_designer` factory.
aidd = gretel.data_designer.new(model_suite="apache-2.0")



---



# Step 4: Start with "Sampler Columns"

Adding a sampler column for unique identifier

In [ ]:
aidd.add_column(
    C.SamplerColumn(
        name="opportunity_id",
        type=P.SamplerType.UUID,
        params=P.UUIDSamplerParams(
            prefix="OPP-",  # Optional: adds a prefix
            short_form=True,  # Optional: uses a shorter format
            uppercase=True  # Optional: uses uppercase letters
        )
    )
)

DataDesigner(
    model_suite: apache-2.0
    sampler_columns: ['opportunity_id']
)

Adding category sampler columns

In [ ]:
aidd.add_column(
    C.SamplerColumn(
        name="opportunity_category",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["New Logo", "Renewal", "Upsell", "Expansion", "Services"],
        )
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="opportunity_stage",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["Stage 0 - New", "Stage 1 - Qualified", "Stage 2 - Technical Demo", "Stage 3 - Proposal/Quote", "Stage 4 - Closed Won","Stage 5 - Closed Lost"],
        )
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="opportunity_stage_detail",
        type=P.SamplerType.SUBCATEGORY,
        params=P.SubcategorySamplerParams(
            category="opportunity_stage",
            values={
                "Stage 0 - New": ["Identify Champion", "Identify Economic Buyer", "Qualify Lead"],
                "Stage 1 - Qualified": ["Identify Pain", "Assign Champion", "Assign Economic Buyer", "Identify Competitor"],
                "Stage 2 - Technical Demo": ["Solution Alignment", "Validate Decision Criteria", "Validate Paper Process and Budget"],
                "Stage 3 - Proposal/Quote": ["Align on Close Plan", "Confirm Procurement Steps", "Finalize Contract Terms"],
                "Stage 4 - Closed Won": ["Hand off to CS","Schedule expansion call internally"],
                "Stage 5 - Closed Lost": ["Track closed lost reason","Schedule review call internally"],

            }
        )
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="opportunity_forecast_category",
        type=P.SamplerType.SUBCATEGORY,
        params=P.SubcategorySamplerParams(
            category="opportunity_stage",
            values={
                "Stage 0 - New": ["New Pipeline"],
                "Stage 1 - Qualified": ["Qualified Pipeline"],
                "Stage 2 - Technical Demo": ["Qualified Pipeline", "Upside", "Commit"],
                "Stage 3 - Proposal/Quote": ["Qualified Pipeline", "Upside", "Commit"],
                "Stage 4 - Closed Won": ["Closed Won"],
                "Stage 5 - Closed Lost": ["Closed Lost"],

            }
        )
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="account_vertical",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["Software", "Consulting", "Financial", "Semiconductors", "Cybersecurity","Manufacturing","Education","Government","AI/ML"],
        )
    )
)
aidd.add_column(
    C.SamplerColumn(
        name="account_segment",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["SMB", "Non-Profit", "Lower Mid-Market", "Upper Mid-Market", "Enterprise","Strategic","Partner"],
        )
    )
)



DataDesigner(
    model_suite: apache-2.0
    person_samplers: ['opportunity_owner']
    sampler_columns: [
        "opportunity_id",
        "opportunity_category",
        "opportunity_stage",
        "opportunity_stage_detail",
        "opportunity_forecast_category",
        "opp_create_date",
        "opp_close_date",
        "quantity",
        "unit_price",
        "account_vertical",
        "account_segment"
    ]
    llm_text_columns: [
        "lead_source",
        "sales_rep_notes",
        "sales_territory",
        "account"
    ]
    expression_columns: ['opportunity_value']
)

Add person sampler column

In [ ]:
aidd.with_person_samplers(
    {
        "opportunity_owner": P.PersonSamplerParams(
        )
    }
)

DataDesigner(
    model_suite: apache-2.0
    person_samplers: ['opportunity_owner']
    sampler_columns: [
        "opportunity_id",
        "opportunity_category",
        "opportunity_stage",
        "opportunity_stage_detail",
        "opportunity_forecast_category",
        "opp_create_date",
        "opp_close_date",
        "quantity",
        "unit_price",
        "account_vertical",
        "account_segment"
    ]
    llm_text_columns: [
        "lead_source",
        "sales_rep_notes",
        "sales_territory",
        "account"
    ]
    expression_columns: ['opportunity_value']
)

Adding date/time sampler columns. Adding a time delta sampler column to represent days until closed won.

In [ ]:
aidd.add_column(
    C.SamplerColumn(
        name="opp_create_date",
        type=P.SamplerType.DATETIME,
        params=P.DatetimeSamplerParams(start="2022-01-01", end="2025-12-31")
    )
)
aidd.add_column(
    C.SamplerColumn(
        name="opp_close_date",
        type=P.SamplerType.TIMEDELTA,
        params=P.TimeDeltaSamplerParams(
            dt_min=1,  # Minimum days
            dt_max=120,  # Maximum days
            reference_column_name="opp_create_date"  # Reference date column
        )
    )
)

DataDesigner(
    model_suite: apache-2.0
    person_samplers: ['opportunity_owner']
    sampler_columns: [
        "opportunity_id",
        "opportunity_category",
        "opportunity_stage",
        "opportunity_stage_detail",
        "opportunity_forecast_category",
        "opp_create_date",
        "opp_close_date",
        "quantity",
        "unit_price",
        "account_vertical",
        "account_segment"
    ]
    llm_text_columns: [
        "lead_source",
        "sales_rep_notes",
        "sales_territory",
        "account"
    ]
    expression_columns: ['opportunity_value']
)

Adding sampler columns for numerical values

In [ ]:
# product quantity, licenses, seats, etc.
# I chose uniform for equal probability of all outcomes, I don't think that would be a problem for this scenario
aidd.add_column(
    C.SamplerColumn(
        name="quantity",
        type=P.SamplerType.UNIFORM,
        params=P.UniformSamplerParams(low=5, high=100000),
        convert_to="int"  # Optional: converts to integer
    )
)

# unit price
aidd.add_column(
    C.SamplerColumn(
        name="unit_price",
        type=P.SamplerType.UNIFORM,
        params=P.UniformSamplerParams(low=17, high=1500),
        convert_to="int"  # Optional: converts to integer
    )
)

# total opp value with expression column
aidd.add_column(
    C.ExpressionColumn(
        name="opportunity_value",
        expr="{{quantity}} * {{unit_price}}"
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="support_tickets",
        type=P.SamplerType.BINOMIAL,
        params=P.BinomialSamplerParams(n=10, p=0.3)
    )
)

DataDesigner(
    model_suite: apache-2.0
    person_samplers: ['opportunity_owner']
    sampler_columns: [
        "opportunity_id",
        "opportunity_category",
        "opportunity_stage",
        "opportunity_stage_detail",
        "opportunity_forecast_category",
        "opp_create_date",
        "opp_close_date",
        "quantity",
        "unit_price",
        "account_vertical",
        "account_segment",
        "support_tickets"
    ]
    llm_text_columns: [
        "lead_source",
        "sales_rep_notes",
        "sales_territory",
        "account"
    ]
    expression_columns: ['opportunity_value']
)



---



# Step 5: Add in LLM-based columns

These columns need to reference sampler columns in the prompt. Otherwise it will return an error.

In [ ]:
aidd.add_column(
    C.LLMTextColumn(
        name="lead_source",
        output_type="text",
        model_alias="text",
        prompt="Generate a key word or phrase that describes a sales or marketing activity. Different marketing or sales activity contribute to {{opportunity_category}}. Make sure to include a diverse variety of digital marketing activites and campaigns across the dataset but only one value in the column. Do not generate completey unique values as the goal is to aggregate rows by this column.",
        system_prompt="You are a GTM strategy analytics professional conducting attribution analysis.",  # Optional
        # output_format=".." # Optional
    )
)

aidd.add_column(
    C.LLMTextColumn(
        name="sales_rep_notes",
        output_type="text",
        model_alias="text",
        prompt="Generate brief description of how {{opportunity_id}} is progressing based on {{opportunity_stage_detail}}. Include prospect or customer feedback, procurement obstacles and additional commentary on sale.",
        system_prompt="You are Regional Sales Manager responsible for generating revenue.",  # Optional
        # output_format=".." # Optional
    )
)


aidd.add_column(
    C.LLMTextColumn(
        name="sales_territory",
        output_type="text",
        model_alias="text",
        prompt="Generate City and Country based on {{opportunity_owner}} and {{account_segment}}.",
        system_prompt="You are a Sales Operations professional responsible for assigning sales territories based on region.",  # Optional
        # output_format=".." # Optional
    )
)

aidd.add_column(
    C.LLMTextColumn(
        name="account",
        output_type="text",
        model_alias="text",
        prompt="Generate a single fictional company name referenced on {{account_vertical}}. Use company names from technology, startups, finance, manufacturing and semiconductor industry",
        system_prompt="You are a SaaS technology customer providing your company name.",  # Optional
        # output_format=".." # Optional
    )
)


DataDesigner(
    model_suite: apache-2.0
    person_samplers: ['opportunity_owner']
    sampler_columns: [
        "opportunity_id",
        "opportunity_category",
        "opportunity_stage",
        "opportunity_stage_detail",
        "opportunity_forecast_category",
        "opp_create_date",
        "opp_close_date",
        "quantity",
        "unit_price",
        "account_vertical",
        "account_segment",
        "support_tickets"
    ]
    llm_text_columns: [
        "lead_source",
        "sales_rep_notes",
        "sales_territory",
        "account"
    ]
    expression_columns: ['opportunity_value']
)



---



# Step 6: Preview the data and iterate

It will take a few iterations, and you may even remember some additional columns that you want to add. The LLM columns will definitely need tweaking so be patient!

In [ ]:
preview = aidd.preview()

[05:01:56] [INFO] 🚀 Generating preview
[05:01:57] [INFO] 🎲 Step 1: Using samplers to generate 13 columns
[05:01:59] [INFO] 🦜 Step 2: Generating text column `lead_source`
[05:02:00] [INFO] 🦜 Step 3: Generating text column `sales_rep_notes`
[05:02:04] [INFO] 🦜 Step 4: Generating text column `sales_territory`
[05:02:05] [INFO] 🦜 Step 5: Generating text column `account`
[05:02:06] [INFO] 💬 Step 6: Rendering expression column `opportunity_value`
[05:02:06] [INFO] 🙈 Step 7: Dropping 1 latent person column
[05:02:06] [INFO] 🧐 Step 8: Evaluating dataset
[05:02:06] [INFO] 🎉 Your dataset preview is ready!


In [ ]:
# pandas dataframe
preview_df.head()

,opportunity_id,opportunity_category,opportunity_stage,opportunity_stage_detail,opportunity_forecast_category,opp_create_date,opp_close_date,quantity,unit_price,lead_source,sales_rep_notes,sales_territory,account,opportunity_value
0,OPP-23079CDA,Upsell,Stage 4 - Closed Won,Hand off to CS,Closed Won,2025-07-03,2025-10-07,75141,133,Marketing Campaign,**OPP-23079CDA Progress Update:**\n\n- **Custo...,City: Harvest\nCountry: USA,KristinTech Solutions,75141 * 133
1,OPP-8BA48C33,Services,Stage 2 - Technical Demo,Solution Alignment,Upside,2022-11-06,2022-11-29,118,956,Touchpoints,**OPP-8BA48C33 Progress Update:**\n\n**Solutio...,City: Lockport\nCountry: USA,TroyTech Solutions,118 * 956
2,OPP-F9DAA689,Expansion,Stage 4 - Closed Won,Schedule expansion call internally,Closed Won,2024-10-14,2024-12-28,18992,17,Expansion Activity,**OPP-F9DAA689 Progress Update**\n\n- **Schedu...,City: Mountain View\n\nCountry: USA,EthanTech Solutions,18992 * 17
3,OPP-065EFB24,Expansion,Stage 4 - Closed Won,Schedule expansion call internally,Closed Won,2025-08-06,2025-08-31,24645,916,Customer Acquisition Campaign,**OPP-065EFB24 Progress Update**\n\n- **Schedu...,City: Reno\nCountry: USA,FlemTech Solutions,24645 * 916
4,OPP-9E05BD62,Upsell,Stage 0 - New,Qualify Lead,New Pipeline,2023-01-20,2023-01-27,41645,105,Digital Marketing Campaign,**OPP-9E05BD62 Progress Description:**\n\n- **...,City: Summit\nCountry: USA,AshleyTech Solutions,41645 * 105


In [ ]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                          ┃ Value                                                                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ opportunity_id                │ OPP-2574980D                                                                    │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ opportunity_category          │ New Logo                                                                        │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ opportunity_stage             │ Stage 2 - Technical Demo                                                        │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ opportunity_stage_detail      │ Solution Alignment                                                              │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ opportunity_forecast_category │ Upside                                                                          │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ opp_create_date               │ 2023-08-14                                                                      │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ opp_close_date                │ 2023-12-05                                                                      │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ quantity                      │ 97549                                                                           │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ unit_price                    │ 215                                                                             │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ account_vertical              │ Semiconductors                                                                  │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ account_segment               │ SMB                                                                             │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ support_tickets               │ 4                                                                               │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ opportunity_value             │ 97549 * 215                                                                     │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ lead_source                   │ Multi-Channel Campaign                                                          │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ sales_rep_notes               │ **OPP-2574980D Progress Description:**                                          │
│                               │                                                                                 │
│                               │ - **Solution Alignment



---



# Step 7: Add evaluation report and scale




This step will add a report evaluating the quality of the synthetic data once it is scaled up.

In [ ]:
aidd.with_evaluation_report()

DataDesigner(
    model_suite: apache-2.0
    person_samplers: ['opportunity_owner']
    sampler_columns: [
        "opportunity_id",
        "opportunity_category",
        "opportunity_stage",
        "opportunity_stage_detail",
        "opportunity_forecast_category",
        "opp_create_date",
        "opp_close_date",
        "quantity",
        "unit_price",
        "account_vertical",
        "account_segment",
        "support_tickets"
    ]
    llm_text_columns: [
        "lead_source",
        "sales_rep_notes",
        "sales_territory",
        "account"
    ]
    expression_columns: ['opportunity_value']
)

Once you are satisfied with the preview data, scale up!

In [ ]:
workflow_run = aidd.create(num_records=10000, name="synth-sales-opportunity-data")

[05:05:29] [INFO] 🚀 Submitting batch workflow


INFO:gretel_client.workflows.builder:▶️ Creating Workflow: w_2xWoKxgT9jBybvT1uRvlFsUJiax
INFO:gretel_client.workflows.builder:▶️ Created Workflow Run: wr_2xWoKxPNA6RxwEM7sSb74NbQqcl
INFO:gretel_client.workflows.builder:🔗 Workflow Run console link: https://console.gretel.ai/workflows/w_2xWoKxgT9jBybvT1uRvlFsUJiax/runs/wr_2xWoKxPNA6RxwEM7sSb74NbQqcl




---

